In [1]:
import numpy as np
import tensorflow as tf

In [103]:
class RNN(tf.Module):
    
    def __init__(self, state_size, batch_size, **kwargs):
        super().__init__(**kwargs)

        # for calculating the next state
        self.w1 = tf.Variable(np.random.rand(state_size + 1, state_size), dtype=tf.float64)
        self.b1 = tf.Variable(np.random.rand(1), dtype=tf.float64)

        # for calculating output
        self.w2 = tf.Variable(np.random.rand(state_size, 1), dtype=tf.float64)
        self.b2 = tf.Variable(np.random.rand(1), dtype=tf.float64)

        self.states = tf.Variable(np.zeros((batch_size, state_size)), shape=(batch_size, state_size))

    # calculate output from current state
    # calcualte next state from current state + input
    def __call__(self, x):
        input_state_stacked = tf.concat([x, self.states], axis=1)
        output = tf.matmul(self.states, self.w2) + self.b2
        self.states = tf.matmul(input_state_stacked, self.w1) + self.b1
        return output
        #return input_state_stacked

    def __str__(self):
        return " ".join([str(i) for i in self.state.numpy()])

* Input shape: [batch_size, sequence_lenght, n_features]
* I will assume n_features=1 to make my life easier

In [104]:
batch_size = 200
sequence_lenght = 10
#n_features = 1
x = np.arange(batch_size * sequence_lenght).reshape((batch_size, sequence_lenght, 1))

In [105]:
rnn = RNN(5, 200)

In [113]:
X = tf.constant(x.copy(), dtype=tf.dtypes.float64)
output = rnn(X[:,1])

In [115]:
output[5:10, :]

<tf.Tensor: shape=(5, 1), dtype=float64, numpy=
array([[140.39654653],
       [167.76176599],
       [195.12698545],
       [222.4922049 ],
       [249.85742436]])>

In [116]:
output = rnn(X[:,1])

In [117]:
output[5:10, :]

<tf.Tensor: shape=(5, 1), dtype=float64, numpy=
array([[276.1941069 ],
       [330.16029183],
       [384.12647675],
       [438.09266168],
       [492.0588466 ]])>